<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/1_2_EVA_CAMS_GLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Verbinden mit der Google Drive und Import der benötigten Module:

In [2]:
# Verbinden mit der Google-Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, sys

In [ ]:
!pip install factor_analyzer
!pip install prophet

In [ ]:
## import modules

import numpy as np
import pandas as pd
import matplotlib

import seaborn as sns

from datetime import datetime
from datetime import timedelta

In [ ]:
latitude = 50.2
longitude = 7.8

seconds_in_day = 24*60*60
seconds_in_year = (365.2425)*seconds_in_day

In [ ]:
# Pfad zur CSV-Datei auf Google Drive
# UVI Index Messwerte
name_Messwerte = 'UVI_Messdaten.csv'
# Globalstrahlungsmesswerte
name_Solar_Messwerte = 'Solys_Messdaten.csv'
# CAMS Meteorologische Vorhersagedaten
name_Vorhersage = 'Vorhersage.csv'
# CAMS Globalstrahlungsvorhersage
name_GlobalVorhersage = 'Cams_solar_radiation.csv'

drive_path = '/content/drive/My Drive/Colab_Notebooks/Clean_Data/'
output_plot_path = '/content/drive/My Drive/Colab_Notebooks/plot_daily_UVI/'